In [ ]:
%run ../../SSB_Spark_tools/data_processing/Utils.py

In [ ]:
skatt_raw = spark.read.path("/kilde/ske/skatt/person/fastsatt/rådata/2019v1.0.1")

In [ ]:
skattemelding_hendelse = skatt_raw\
                        .select('hendelse.*', 'skattemeldingUtflatet')\
                        .withColumnRenamed('registreringstidspunkt', 'hendelsetidspunkt')\
                        .withColumn('hendelsetidspunkt', F.to_timestamp('hendelsetidspunkt', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
skattemelding_utflatet = skattemelding_hendelse\
                        .select('identifikator', 'gjelderPeriode', 'hendelsetidspunkt', 'hendelsetype', 'skattemeldingUtflatet.*')\
                        .withColumn('registreringstidspunkt', F.to_timestamp('registreringstidspunkt', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))\
                        .cache()
                        
skattemelding_tverrsnitt =  cross_sectional(skattemelding_utflatet, event_var = 'hendelsetidspunkt', event_id = ['identifikator', 'gjelderPeriode'])


In [ ]:
tverrsnitt_filtrert = skattemelding_tverrsnitt\
                        .filter(F.col('hendelsetype')!='slettet')\
                        .filter(F.col('skjermet')==False)\
                        .filter(F.col('identifikator')==F.col('personidentifikator'))

In [ ]:
so_dict_testtverrsnitt = unpack_parquet(tverrsnitt_filtrert, rootdf=True, rootvar=['personidentifikator'])
for k in so_dict_testtverrsnitt.keys():
    print(k)

In [ ]:
so_dict_testtverrsnitt['konto'].toPandas()